In [ ]:
!pip install langchain transformers sentence-transformers faiss-cpu beautifulsoup4 requests

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install -U langchain-huggingface

In [ ]:
import requests
import json
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
from google.colab import files
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from sentence_transformers import SentenceTransformer, util

In [ ]:
uploaded=files.upload()

Saving Talos_Vulnerability_QA (1).xlsx to Talos_Vulnerability_QA (1).xlsx


In [ ]:
df=pd.read_excel("Vulnerabilities_QA.xlsx")

In [ ]:
df

,Report Name,Question,Answer
0,TALOS-2024-2113,What type of flaw exists in the nvJPEG2000 lib...,A heap-based buffer overflow occurs due to imp...
1,TALOS-2024-1965,What class of vulnerability is reported in Mic...,Multiple out-of-bounds read vulnerabilities ha...
2,TALOS-2024-2131,Which component of catdoc is affected and what...,The OLE Document File Allocation Table (FAT) P...
3,TALOS-2024-2023,What vulnerability allows remote misuse on the...,An OS command injection flaw exists in iptable...
4,TALOS-2024-2069,What type of flaw affects GNOME’s libgsf in ve...,An integer overflow in the Compound Document B...
5,TALOS-2024-2005,What potential consequence can a crafted Ether...,It can lead to remote code execution by sendin...
6,TALOS-2024-1983,What information leak occurs in the LevelOne W...,Sensitive data is transmitted in cleartext ove...
7,TALOS-2024-2051,Which interface in Wavlink’s openvpn.cgi is ex...,The openvpn_client_setup() interface suffers f...
8,TALOS-2024-2013,Which subcomponent in NVIDIA’s driver has a cr...,The Shader Functionality in NVIDIA D3D10 Drive...
9,TALOS-2024-2122,What vulnerability in Allen-Bradley MicroLogix...,An unauthenticated write-what-where condition ...


In [ ]:
#Step 3: Load LLM
model_id = "meta-llama/Llama-3.2-3B-Instruct"
hf_token = "" # your token
similarity_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_token
)

def generate(prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def build_vectorstore(docs, chunk_count):
    return FAISS.from_documents(docs[:chunk_count], embedding)

In [ ]:
def scrape_page(url):
  try:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    target_section = soup.find("div", class_="report")
    if not target_section:
        print(f"Class report not found in {url}")
        return ""

    return target_section.get_text(separator="\n", strip=True)

  except Exception as e:
    print(f"Failed to process report: {e}")
    return []

def get_chunks_from_url(url, chunk_size=350, chunk_overlap=0):
    raw_text = scrape_page(url)

    if not raw_text.strip():
        print(f"No text found at {url}")
        return []

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(raw_text)
    chunked_docs = [Document(page_content=chunk, metadata={"source": url}) for chunk in chunks]
    return chunked_docs


def get_chunk_similarity_scores(db,query, chunk_count):
   results_with_scores = db.similarity_search_with_score(query, k=chunk_count)
   return results_with_scores

def get_semantic_similarity(rag_response, without_rag_response):
    embeddings = similarity_model.encode([rag_response, without_rag_response], convert_to_tensor=True)
    sim_score = float(util.pytorch_cos_sim(embeddings[0], embeddings[1])[0])
    return sim_score

def print_semantic_similarity(rag_score, nonrag_score, rag_response, chunk_count):
    print(f"RAG Similarity: {rag_score:.4f}\n")
    print(f"Non-RAG Similarity: {nonrag_score:.4f}\n")

def print_individual_chunk_similarity(results_with_scores):
    print()
    print("*" * 500)
    for i, (doc, score) in enumerate(results_with_scores, 1):
        print()
        print(f"Chunk {i} | Similarity Score: {score:.4f}")
        print("Chunk Text:", doc.page_content)
        print("Metadata:", doc.metadata)
        print("*" * 500)
        print()

def get_prompt(context, query):
      return f"""Use the context below to answer the question.

      Only answer if you are confident. If you are unsure or you feel that you need more information to answer confidently, say: "I am unsure and need more information."

Context:
{context}

Question: {query}

Answer:"""

def evaluate_rag_with_uncertainty_prompt(query, chatgpt_answer, db, chunk_docs, current_chunk_count):
    chunk_number = current_chunk_count
    print(f"Chunks used as context: {chunk_number}")
    retriever = db.as_retriever(search_kwargs={"k": chunk_number})

    relevant_docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    rag_prompt = get_prompt(context, query)

    rag_response = generate(rag_prompt)
    non_rag_response = generate(query)

    rag_response_clean = rag_response.split("Answer:")[-1].strip()

    is_uncertain = "i am unsure" in rag_response_clean.lower()
    print()
    print(f"Is RAG response uncertain? {is_uncertain}")

    print()
    print(f"\n RAG Response: \n{rag_response_clean}")
    print()

    parts = non_rag_response.strip().split("\n\n", 1)
    non_rag_response_clean = parts[1].strip() if len(parts) > 1 else non_rag_response.strip()
    print()
    print(f"\n Non-RAG Response Clean: \n{non_rag_response_clean}")
    print()

    rag_score = get_semantic_similarity(chatgpt_answer, rag_response_clean)
    nonrag_score = get_semantic_similarity(chatgpt_answer, non_rag_response_clean)

    print_semantic_similarity(rag_score, nonrag_score, rag_response, chunk_number)
    results_with_scores = get_chunk_similarity_scores(db, query, chunk_number)

    print(results_with_scores)

    if is_uncertain:
      print(f"\nModel is unsure. Increasing chunk count and retrying...\n")
      print('%'*1000)

    return is_uncertain, rag_response_clean, non_rag_response_clean, rag_score, nonrag_score

In [ ]:
attempt_logs = []

for idx, row in df.iterrows():
  initial_chunk_count = 10
  current_chunk_count = initial_chunk_count
  add_chunk_count = 5
  report_name = row['Report Name']
  query = row['Question']
  chatGPT_answer = row['Answer']
  url = f"https://talosintelligence.com/vulnerability_reports/{report_name}"

  chunks = get_chunks_from_url(url)

  if not chunks:
      print(f"No chunks found for {report_name}")
      continue

  db = build_vectorstore(chunks, initial_chunk_count)

  while True:
      is_uncertain, rag_response_clean, non_rag_response_clean, rag_score, nonrag_score = evaluate_rag_with_uncertainty_prompt(query, chatGPT_answer, db, chunks, current_chunk_count)

      attempt_logs.append({
          "report_name": report_name,
          "query": query,
          "chatgpt_answer": chatGPT_answer,
          "chunk_count": current_chunk_count,
          "rag_answer": rag_response_clean,
          "non_rag_answer": non_rag_response_clean,
          "rag_similarity": rag_score,
          "non_rag_similarity": nonrag_score,
          "is_uncertain": is_uncertain
      })

      if not is_uncertain:
          outer_loop_var=False
          break


      current_chunk_count += add_chunk_count
      more_docs = chunks[current_chunk_count - add_chunk_count:current_chunk_count]

      if more_docs:
          db.add_documents(more_docs)
      else:
          print("No more chunks available to add.")
          break

      print(f"Current chunk count: {current_chunk_count}")

In [ ]:
attempt_logs

In [ ]:
df_attemp_logs = pd.DataFrame(attempt_logs)
df_attemp_logs.to_csv("rag_uncertainty_prompt_attempt_logs.csv", index=False)

In [ ]:
files.download("rag_uncertainty_prompt_attempt_logs.csv")